<a href="https://colab.research.google.com/github/nattaran/HealthTequity-LLM/blob/main/VoicePipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Google Drive

In [7]:
# === Simple Google Colab Setup (Clone + Pull + Install) ===

# 🔧 Customize these
project_name = "HealthTequity-LLM"
repo_url = "https://github.com/nattaran/HealthTequity-LLM.git"

import os, shutil

project_path = f"/content/{project_name}"

# 1️⃣ Clean up accidental nested /content/content
if os.path.exists("/content/content"):
    shutil.rmtree("/content/content")

# 2️⃣ Clone or update repository
if not os.path.exists(project_path):
    print(f"🔹 Cloning repository into {project_path} ...")
    !git clone {repo_url} {project_path}
else:
    print(f"🔹 Repository already exists at {project_path}. Pulling latest changes...")
    %cd {project_path}
    !git fetch origin
    !git pull

# 3️⃣ Move into the project folder
%cd {project_path}

# 4️⃣ Auto-install dependencies if requirements.txt exists
if os.path.exists("requirements.txt"):
    print("🔹 Installing dependencies from requirements.txt ...")
    !pip install -r requirements.txt
else:
    print("ℹ️ No requirements.txt found — skipping dependency installation.")

# 5️⃣ Confirm setup
print("\n✅ Project setup complete!")
!pwd
!ls -la


🔹 Cloning repository into /content/HealthTequity-LLM ...
Cloning into '/content/HealthTequity-LLM'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 59 (delta 22), reused 23 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (59/59), 1.67 MiB | 10.69 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/content/HealthTequity-LLM
🔹 Installing dependencies from requirements.txt ...
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-dt4cierj
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-dt4cierj
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.6 MB/s eta 0:00

In [9]:
# 3️⃣ Verify existing data folders
# import os
# expected_dirs = [
#     "data/Spanish_audio",
#     "data/synthetic_csv",
#     "results/llm_outputs",
#     "results/evaluation_metrics",
#     "results/tts_audio",
# ]
# for d in expected_dirs:
#     if not os.path.exists(d):
#         os.makedirs(d)
#         print(f"📁 Created missing folder: {d}")
#     else:
#         print(f"✅ Folder found: {d}")

# print("\n✅ Environment setup complete. Repository ready to use.")

In [6]:
! cd health

/content


# **Step 1 : Path Configuration**

In [5]:
# ==========================================================
# 📁 PATH CONFIGURATION (Optimized for Google Colab)
# ==========================================================
import os
from pathlib import Path

# --- Define your main project root path ---
PROJECT_ROOT = Path("/content/HealthTequity-LLM")

# --- Ensure we are inside the project root ---
if Path.cwd() != PROJECT_ROOT:
    os.chdir(PROJECT_ROOT)
print(f"📂 Current working directory: {Path.cwd()}")

# --- Define key subdirectories ---
DATA_DIR   = PROJECT_ROOT / "data"
RESULTS_DIR = PROJECT_ROOT / "results"
NOTEBOOKS_DIR = PROJECT_ROOT / "notebooks"

# --- Define specific folders for pipeline I/O ---
AUDIO_DIR = DATA_DIR / "Spanish_audio"
CSV_DIR   = DATA_DIR / "synthetic_csv"
LLM_OUT   = RESULTS_DIR / "llm_outputs"
EVAL_DIR  = RESULTS_DIR / "evaluation_metrics"
TTS_DIR   = RESULTS_DIR / "tts_audio"

# --- Create any missing directories (only for outputs) ---
for path in [LLM_OUT, EVAL_DIR, TTS_DIR]:
    path.mkdir(parents=True, exist_ok=True)

# --- Print summary for verification ---
print("✅ Folder configuration complete:")
print(f"   Audio dir: {AUDIO_DIR}")
print(f"   CSV dir:   {CSV_DIR}")
print(f"   LLM out:   {LLM_OUT}")
print(f"   Eval dir:  {EVAL_DIR}")
print(f"   TTS dir:   {TTS_DIR}")


FileNotFoundError: [Errno 2] No such file or directory: '/content/HealthTequity-LLM'

In [12]:
# ==========================================================
# 🔍 VERIFY AVAILABLE FILES
# ==========================================================
# from pathlib import Path

# def list_files_in_folder(folder: Path, pattern: str):
#     files = sorted(folder.glob(pattern))
#     if files:
#         print(f"✅ Found {len(files)} file(s) in {folder}:")
#         for f in files[:5]:
#             print("   •", f.name)
#         if len(files) > 5:
#             print(f"   ...and {len(files) - 5} more.\n")
#     else:
#         print(f"⚠️ No files found in {folder}. Check your repo or path.\n")
#     return files

# # --- Check your Spanish audio files ---
# audio_files = list_files_in_folder(AUDIO_DIR, "*.wav")

# # --- Check your synthetic blood pressure CSV files ---
# csv_files = list_files_in_folder(CSV_DIR, "*.csv")

# # --- Sanity checks ---
# assert len(csv_files) > 0, "❌ No CSV files found in data/synthetic_csv/"
# print("\n✅ Folder verification complete.")


# Load OpenAI API Key (From secretes)

In [2]:
# ==========================================================
# 🔑 OPENAI CLIENT (COLAB-SECURE)
# ==========================================================
import os
from getpass import getpass
from openai import OpenAI

# Ask for key only if it isn't set yet
if "OPENAI_API_KEY" not in os.environ or not os.environ["OPENAI_API_KEY"]:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OPENAI_API_KEY: ")

client = OpenAI()
print("✅ OpenAI client initialized successfully.")


Enter your OPENAI_API_KEY: ··········
✅ OpenAI client initialized successfully.


In [7]:
import whisper
print("✅ Whisper module path:", whisper.__file__)
print("✅ Available attributes:", [m for m in dir(whisper) if "load" in m])

✅ Whisper module path: /usr/local/lib/python3.11/dist-packages/whisper/__init__.py
✅ Available attributes: ['__loader__', '_download', 'load_audio', 'load_model']


In [1]:
# !pip install git+https://github.com/openai/whisper.git

In [18]:
# ==========================================================
# 🎙️ STEP 4 — TRANSCRIBE (SPANISH) ➜ TRANSLATE (ENGLISH)
# ==========================================================
import os
import pandas as pd
import whisper
from pathlib import Path

# --- 1️⃣ Transcription Function ---
def transcribe_spanish_audio(model, audio_path: Path):
    print(f"🎧 Transcribing: {audio_path.name}")
    result = model.transcribe(str(audio_path), language="es", task="transcribe", verbose=False)
    return result["text"].strip(), result.get("language", "unknown")

# --- 2️⃣ Translation Function ---
def translate_spanish_to_english(spanish_text: str) -> str:
    """Translate Spanish transcription to English."""
    prompt = f"Translate the following Spanish medical transcription into clear English:\n\n{spanish_text}"
    result = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[{"role": "user", "content": prompt}]
    )
    return result.choices[0].message.content.strip()

# --- 3️⃣ Full Pipeline ---
def process_and_translate_audio(audio_folder: Path, output_csv: Path):
    """
    Runs Whisper ASR on all .wav files in audio_folder,
    translates Spanish → English, and saves to CSV.
    """
    model = whisper.load_model("base")
    all_results = []

    audio_files = sorted([f for f in os.listdir(audio_folder) if f.endswith(".wav")])

    print("\n🎯 STARTING SPANISH TRANSCRIPTION + TRANSLATION\n" + "="*60)
    for i, audio_file in enumerate(audio_files, 1):
        audio_path = audio_folder / audio_file
        if not audio_path.exists():
            print(f"⚠️ {audio_file} not found, skipping...")
            continue

        spanish_text, detected_lang = transcribe_spanish_audio(model, audio_path)
        english_text = translate_spanish_to_english(spanish_text)

        all_results.append({
            "audio_file": audio_file,
            "spanish_transcription": spanish_text,
            "english_translation": english_text,
            "language_detected": detected_lang
        })

        print(f"\n[{i}] {audio_file}")
        print(f"🇪🇸 {spanish_text}")
        print(f"🇬🇧 {english_text}")

    df = pd.DataFrame(all_results)
    df.to_csv(output_csv, index=False, encoding="utf-8-sig")
    print(f"\n✅ Transcriptions + translations saved to {output_csv}")
    return df


In [20]:
# ==========================================================
# 🚀 RUN TRANSCRIPTION + TRANSLATION PIPELINE
# ==========================================================
output_csv = LLM_OUT / "asr_translation_results.csv"
df_results = process_and_translate_audio(AUDIO_DIR, output_csv)



🎯 STARTING SPANISH TRANSCRIPTION + TRANSLATION
🎧 Transcribing: q1_es.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
100%|██████████| 458/458 [00:02<00:00, 158.30frames/s]



[1] q1_es.wav
🇪🇸 ¿Cuáles son mis presiones arteriales histólica y diastólica hoy?
🇬🇧 What are my systolic and diastolic blood pressures today?
🎧 Transcribing: q2_es.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
100%|██████████| 470/470 [00:04<00:00, 105.30frames/s]



[2] q2_es.wav
🇪🇸 ¿Cuáles fueron mis valores de presión arterial durante la última semana?
🇬🇧 What were my blood pressure readings over the last week?
🎧 Transcribing: q3_es.wav


...


[4] q4_es.wav
🇪🇸 ¿Cuáles son los rango normales para una persona como yo?
🇬🇧 What are the normal ranges for a person like me?
🎧 Transcribing: q5_es.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
100%|██████████| 328/328 [00:03<00:00, 96.74frames/s]



[5] q5_es.wav
🇪🇸 ¿Cuál era mi presión arterial el 10 de octubre?
🇬🇧 What was my blood pressure on October 10th?
🎧 Transcribing: q6_es.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
100%|██████████| 1574/1574 [00:04<00:00, 339.54frames/s]



[6] q6_es.wav
🇪🇸 ¿En qué día mi presión arterial excedió los niveles normales? Compare mi presión arterial promedio en la primera semana y la última semana de este mes. ¿Cuál fue mi presión arterial diastólica más baja este mes?
🇬🇧 On what day did my blood pressure exceed normal levels? Compare my average blood pressure in the first week and the last week of this month. What was my lowest diastolic blood pressure this month?

✅ Transcriptions + translations saved to /content/HealthTequity-LLM/results/llm_outputs/asr_translation_results.csv


In [21]:
# ==========================================================
# 🧮 ASR PERFORMANCE EVALUATION (WER, CER, SER)
# ==========================================================
import pandas as pd
import Levenshtein
from jiwer import process_words
from pathlib import Path

def compute_cer(reference: str, hypothesis: str) -> float:
    reference, hypothesis = reference.strip(), hypothesis.strip()
    if not reference:
        return 1.0 if hypothesis else 0.0
    return Levenshtein.distance(reference, hypothesis) / len(reference)

def compute_sentence_error(reference: str, hypothesis: str) -> int:
    return 0 if reference.strip() == hypothesis.strip() else 1

def evaluate_asr_performance(ground_truth_csv: Path, transcribed_csv: Path, output_csv: Path):
    gt_df = pd.read_csv(ground_truth_csv)
    tr_df = pd.read_csv(transcribed_csv)

    gt_df.columns = [c.lower().strip() for c in gt_df.columns]
    tr_df.columns = [c.lower().strip() for c in tr_df.columns]

    df = pd.merge(gt_df, tr_df, on="audio_file", how="inner")

    results = []
    print(f"\n🎯 Evaluating {len(df)} audio files for ASR performance...\n")

    for _, row in df.iterrows():
        ref, hyp = str(row["ground_truth"]), str(row["spanish_transcription"])
        m = process_words(ref, hyp)
        wer_score = round(m.wer, 4)
        cer = round(compute_cer(ref, hyp), 4)
        ser = compute_sentence_error(ref, hyp)

        results.append({
            "audio_file": row["audio_file"],
            "WER": wer_score,
            "Substitutions": m.substitutions,
            "Deletions": m.deletions,
            "Insertions": m.insertions,
            "CER": cer,
            "SER": ser
        })
        print(f"🎧 {row['audio_file']} → WER: {wer_score}, CER: {cer}, SER: {ser}")

    out_df = pd.DataFrame(results)
    out_df.to_csv(output_csv, index=False)
    print(f"\n✅ ASR metrics saved to: {output_csv}")
    return out_df


In [23]:
ground_truth_csv = CSV_DIR / "ground_truth.csv"
transcribed_csv  = LLM_OUT / "asr_translation_results.csv"
output_metrics   = EVAL_DIR / "asr_metrics.csv"

df_metrics = evaluate_asr_performance(ground_truth_csv, transcribed_csv, output_metrics)


🎯 Evaluating 6 audio files for ASR performance...

🎧 q1_es.wav → WER: 0.1111, CER: 0.0156, SER: 1
🎧 q2_es.wav → WER: 0.0, CER: 0.0, SER: 0
🎧 q3_es.wav → WER: 0.0, CER: 0.0, SER: 0
🎧 q4_es.wav → WER: 0.1, CER: 0.0175, SER: 1
🎧 q5_es.wav → WER: 0.0, CER: 0.0, SER: 0
🎧 q6_es.wav → WER: 0.0, CER: 0.0, SER: 0

✅ ASR metrics saved to: /content/HealthTequity-LLM/results/evaluation_metrics/asr_metrics.csv


In [ ]:
# LLM Section

# ================================================================
# 🤖 STEP — GPT BLOOD PRESSURE ANALYSIS + TRANSLATION + TTS
# ================================================================
import json, re, os
from openai import OpenAI
from pathlib import Path

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# --- 🧠 Updated System Instruction ---
SYSTEM = """
You are a careful and detail-oriented data analyst.

You are given a synthetic blood pressure dataset in CSV format. It contains readings for one individual over the last 30 consecutive days, with the following columns:

- date
- age
- sex
- systolic_mmHg
- diastolic_mmHg

Use only the data in the CSV to answer all questions, except when normal blood pressure ranges are requested — in those cases, you may use external references but must cite your source.

---

🧠 Interpretation Guidelines:

- "Today" refers to the most recent date in the dataset.
- "Yesterday" means the most recent date before "today", based on available data.
- Phrases like "last week" or "last month" refer to calendar-based timeframes (e.g., the 7 or 30 days before "today"), not just row counts.
- If a question refers to a specific date or date range that is not present in the dataset, clearly state that the data is unavailable.
- Use conversational date formats like “October 12” or “October 12 to 15” — avoid numeric formats like “10/12/2025”.

---

💬 Answer Style:

- Write in natural, conversational English.
- Address the user directly using “you” (e.g., “Your blood pressure was…”).

---

✅ Response Format:

Return all answers in the following JSON format:

{ "answer": "<English answer>", "computed_fields": { "numeric values used" } }

---

📌 Example Questions and Answers:

These illustrate tone and structure only. Actual answers must be based on the CSV data.

Q: What are my systolic_mmHg and diastolic_mmHg blood pressures today?
A: Your systolic blood pressure was [xx] mm Hg and your diastolic pressure was [yy] mm Hg today.

Q: What were the values over the last week?
A: Over the last 7 days, your systolic pressure averaged [xx] mm Hg and your diastolic pressure averaged [yy] mm Hg.

Q: What is the trend of my blood pressure?
A: Your blood pressure has shown a gradual increase in systolic values over the last 30 days, while your diastolic readings have remained stable.

Q: What are the normal ranges for a person like me?
A: Based on your age ([age from dataset] years) and sex ([male/female]), typical blood pressure values are approximately [xx/yy] mm Hg, according to [name and link to the external source].
"""

# --- 1️⃣ Ask GPT to analyze CSV ---
def ask_gpt(question_en: str, csv_block: str):
    """Ask GPT to analyze CSV data and return JSON-formatted answers."""
    user = f"CSV data:\n{csv_block}\n\nQUESTION:\n{question_en}"
    resp = client.chat.completions.create(
        model="gpt-4o",
        temperature=0.1,
        messages=[
            {"role": "system", "content": SYSTEM},
            {"role": "user", "content": user}
        ]
    ).choices[0].message.content

    # Extract JSON safely
    clean = re.sub(r"^```json|```$", "", resp.strip(), flags=re.M | re.I)
    start, end = clean.find("{"), clean.rfind("}")
    return json.loads(clean[start:end+1])

# --- 2️⃣ English → Spanish translation ---
def translate_to_spanish(english_text: str) -> str:
    """Translate GPT's English output into clear, neutral Spanish."""
    prompt = f"Translate this English medical answer into clear, neutral Spanish:\n{english_text}"
    return client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[{"role": "user", "content": prompt}]
    ).choices[0].message.content.strip()

# --- 3️⃣ Spanish Text-to-Speech (TTS) ---
def text_to_speech_spanish(text: str, out_path: Path, voice="alloy") -> Path:
    """Generate a Spanish TTS audio file for the final translated response."""
    out_path.parent.mkdir(parents=True, exist_ok=True)
    with client.audio.speech.with_streaming_response.create(
        model="gpt-4o-mini-tts", voice=voice, input=text
    ) as response:
        response.stream_to_file(str(out_path))
    print(f"🔊 Saved Spanish audio: {out_path}")
    return out_path


In [2]:
# ================================================================
# 🚀 STEP — FULL PIPELINE: ASR → EVALUATION → LLM → TRANSLATION → TTS
# ================================================================
import json, pandas as pd, os
from pathlib import Path


def run_full_pipeline(csv_path: Path, audio_folder: Path):
    """
    Runs the complete HealthTequity pipeline:
    1. Transcribes and translates Spanish audio → English.
    2. Evaluates ASR using ground truth (WER, CER, SER).
    3. Uses LLM to analyze blood pressure dataset and answer each English question.
    4. Translates answers to Spanish and generates Spanish TTS.
    5. Saves all results to /results folders.
    """
    # --- Step 1️⃣: Transcribe + Translate ---
    trans_csv = LLM_OUT / "audio_translations.csv"
    audio_files = sorted([f for f in os.listdir(audio_folder) if f.endswith(".wav")])
    trans_df = process_and_translate_audio(audio_folder, audio_files, trans_csv)

    # --- Step 2️⃣: Evaluate ASR (WER, CER, SER) ---
    gt_csv = CSV_DIR / "ground_truth.csv"
    asr_csv = EVAL_DIR / "asr_metrics.csv"
    asr_df = evaluate_asr_performance(gt_csv, trans_csv, asr_csv)

    # --- Step 3️⃣: Load synthetic blood pressure CSV ---
    df_bp = pd.read_csv(csv_path)
    csv_block = df_bp.to_csv(index=False)

    # --- Step 4️⃣: For each question, ask GPT and generate responses ---
    results = []
    print("\n🎯 STARTING GPT QUESTION-ANSWER PIPELINE\n" + "=" * 60)

    for i, row in trans_df.iterrows():
        q_num = i + 1
        q_en = row["english_translation"]
        print(f"\n🔹 Q{q_num}: {q_en}")

        try:
            # LLM analysis
            ans = ask_gpt(q_en, csv_block)
            ans_en = ans.get("answer", "").strip()

            # Translate & synthesize
            ans_es = translate_to_spanish(ans_en)
            audio_answer_file = TTS_DIR / f"answer_{q_num}_es.wav"
            text_to_speech_spanish(ans_es, audio_answer_file)

            # Append all outputs
            results.append({
                "question_number": q_num,
                "audio_file_in": row["audio_file"],
                "spanish_question": row["spanish_transcription"],
                "english_question": q_en,
                "english_answer": ans_en,
                "spanish_answer": ans_es,
                "audio_answer_file": str(audio_answer_file),
                "computed_fields": json.dumps(ans.get("computed_fields", {}))
            })

            print(f"✅ Completed Q{q_num}")

        except Exception as e:
            print(f"❌ Error Q{q_num}: {e}")

    # --- Step 5️⃣: Save Final Results ---
    final_csv = LLM_OUT / "final_pipeline_results.csv"
    pd.DataFrame(results).to_csv(final_csv, index=False)
    print(f"\n✅ Full pipeline completed successfully!")
    print(f"📁 Final results saved to: {final_csv}")
    return results


In [3]:
bp_csv = CSV_DIR / "synthetic_bp_one_person.csv"
run_full_pipeline(bp_csv, AUDIO_DIR)


NameError: name 'CSV_DIR' is not defined

In [4]:
CSV_DIR


NameError: name 'CSV_DIR' is not defined

# Define Paths and Create Folders

In [89]:
# import json, re, pandas as pd, whisper, Levenshtein
# from openai import OpenAI
# from jiwer import wer, mer, wil, process_words
# import warnings
# warnings.filterwarnings("ignore")
# import os

# BASE_PATH = "/content/drive/MyDrive/health-tequity-case"

# # --- Define key folders ---
# AUDIO_INPUT_FOLDER = os.path.join(BASE_PATH, "Input_Audio_Files")        # Spanish question audio files
# AUDIO_OUTPUT_FOLDER = os.path.join(BASE_PATH, "Data", "audio_out")       # Spanish TTS answers
# CSV_OUTPUT_FOLDER = os.path.join(BASE_PATH, "Data", "csv_results")       # WER, CER, SER + pipeline outputs
# BP_DATA_FOLDER = os.path.join(BASE_PATH, "Data", "BloodPressure")        # Blood pressure dataset

# # --- Create required folders if they don’t exist ---
# for folder in [AUDIO_OUTPUT_FOLDER, CSV_OUTPUT_FOLDER, BP_DATA_FOLDER]:
#     os.makedirs(folder, exist_ok=True)

# # --- Validate Input Audio Folder ---
# if not os.path.exists(AUDIO_INPUT_FOLDER):
#     raise FileNotFoundError(f"❌ Input folder not found: {AUDIO_INPUT_FOLDER}")

# # --- Collect available audio files ---
# audio_files = [f for f in os.listdir(AUDIO_INPUT_FOLDER) if f.lower().endswith(('.wav', '.mp3', '.m4a'))]
# if not audio_files:
#     raise ValueError(f"❌ No audio files found in {AUDIO_INPUT_FOLDER}")

# print(f"✅ Found {len(audio_files)} Spanish audio file(s): {audio_files}")

# # --- Blood Pressure dataset check ---
# csv_path = os.path.join(BP_DATA_FOLDER, "synthetic_bp_one_person.csv")

# if not os.path.exists(csv_path):
#     print(f"⚠️ Blood pressure dataset not found at:\n   {csv_path}")
#     print("👉 Please upload your synthetic_bp_one_person.csv to this folder before running the pipeline.")
# else:
#     print(f"✅ Found blood pressure dataset: {csv_path}")

# # --- Initialize OpenAI client ---
# client = OpenAI(api_key=api_key)
# print("✅ OpenAI client initialized successfully.")


# ASR Transcrioption Generation Using openAI Whisper Model
**Audio -> Transcription -> English Transcription**

In [ ]:
def transcribe_spanish_audio(model, audio_path):
    print(f"🎧 Transcribing: {audio_path}")
    result = model.transcribe(audio_path, language="spanish", task="transcribe", verbose=False)
    return result["text"].strip(), result["language"]

def translate_spanish_to_english(spanish_text: str) -> str:
    """Translate Spanish transcription to English."""
    prompt = f"Translate the following Spanish medical question into clear English:\n\n{spanish_text}"
    result = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    return result.choices[0].message.content.strip()

def process_and_translate_audio(audio_folder, audio_files, output_csv):
    model = whisper.load_model("base")
    all_results = []

    print("\n🎯 STARTING SPANISH TRANSCRIPTION + TRANSLATION\n" + "="*60)
    for i, audio_file in enumerate(audio_files, 1):
        audio_path = os.path.join(audio_folder, audio_file)
        if not os.path.exists(audio_path):
            print(f"⚠️ {audio_file} not found, skipping...")
            continue

        spanish_text, detected_lang = transcribe_spanish_audio(model, audio_path)
        english_text = translate_spanish_to_english(spanish_text)

        all_results.append({
            "audio_file": audio_file,
            "spanish_transcription": spanish_text,
            "english_translation": english_text,
            "language_detected": detected_lang
        })

        print(f"\n[{i}] {audio_file}")
        print(f"🇪🇸 {spanish_text}")
        print(f"🇬🇧 {english_text}")

    df = pd.DataFrame(all_results)
    df.to_csv(output_csv, index=False)
    print(f"\n✅ Transcriptions + translations saved to {output_csv}")
    return df


# ASR Evaluation (WER, CER, SER)


In [ ]:
def compute_cer(reference: str, hypothesis: str) -> float:
    reference, hypothesis = reference.strip(), hypothesis.strip()
    if not reference:
        return 1.0 if hypothesis else 0.0
    return Levenshtein.distance(reference, hypothesis) / len(reference)

def compute_sentence_error(reference: str, hypothesis: str) -> int:
    return 0 if reference.strip() == hypothesis.strip() else 1

def evaluate_asr_performance(ground_truth_csv, transcribed_csv, output_csv):
    gt_df = pd.read_csv(ground_truth_csv)
    tr_df = pd.read_csv(transcribed_csv)
    gt_df.columns = [c.lower().strip() for c in gt_df.columns]
    tr_df.columns = [c.lower().strip() for c in tr_df.columns]
    df = pd.merge(gt_df, tr_df, on="audio_file", how="inner")

    results = []
    print(f"\n🎯 Evaluating {len(df)} files for ASR performance...\n")
    for _, row in df.iterrows():
        ref, hyp = str(row["ground_truth"]), str(row["spanish_transcription"])
        m = process_words(ref, hyp)
        wer_score = round(m.wer, 4)
        subs, dels, ins = m.substitutions, m.deletions, m.insertions
        cer = round(compute_cer(ref, hyp), 4)
        ser = compute_sentence_error(ref, hyp)
        results.append({
            "audio_file": row["audio_file"],
            "WER": wer_score, "Substitutions": subs,
            "Deletions": dels, "Insertions": ins,
            "CER": cer, "SER": ser
        })
        print(f"🎧 {row['audio_file']} → WER: {wer_score}, CER: {cer}, SER: {ser}")

    out_df = pd.DataFrame(results)
    out_df.to_csv(output_csv, index=False)
    print(f"\n✅ ASR metrics saved to: {output_csv}")
    return out_df

# *GPT Data Analysis + Translation + TTS*

In [ ]:
# ================================================================
# 3️⃣ GPT DATA ANALYSIS + TRANSLATION + TTS
# ================================================================
SYSTEM = """
You are a careful data analyst.
You receive a synthetic blood pressure dataset with columns: date, age, sex, systolic, diastolic.
Do ALL analysis yourself using ONLY the CSV provided.
Answer questions like: daily readings, averages, trends, comparisons, etc.
Return JSON:
{ "answer": "<English answer>", "computed_fields": { "numeric values used" } }
"""

def ask_gpt(question_en, csv_block):
    user = f"CSV data:\n{csv_block}\n\nQUESTION:\n{question_en}"
    resp = client.chat.completions.create(
        model="gpt-4o",
        temperature=0,
        messages=[{"role": "system", "content": SYSTEM}, {"role": "user", "content": user}]
    ).choices[0].message.content
    clean = re.sub(r"^```json|```$", "", resp.strip(), flags=re.M | re.I)
    start, end = clean.find("{"), clean.rfind("}")
    return json.loads(clean[start:end+1])

def translate_to_spanish(english_text):
    prompt = f"Translate this English medical answer into clear, neutral Spanish:\n{english_text}"
    return client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    ).choices[0].message.content.strip()

def text_to_speech_spanish(text, filename, voice="alloy"):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with client.audio.speech.with_streaming_response.create(
        model="gpt-4o-mini-tts", voice=voice, input=text
    ) as response:
        response.stream_to_file(filename)
    print(f"🔊 Saved Spanish audio: {filename}")
    return filename


# Main PIPELINE

In [ ]:
# ================================================================
# 4️⃣ MAIN PIPELINE
# ================================================================
def run_full_pipeline(csv_path, audio_folder, audio_files):
    # Step 1 — Transcribe and Translate Spanish Audio
    trans_csv = os.path.join(CSV_OUTPUT_FOLDER, "audio_translations.csv")
    trans_df = process_and_translate_audio(audio_folder, audio_files, trans_csv)

    # Step 2 — Evaluate ASR (WER, CER, SER)
    gt_csv = os.path.join(audio_folder, "ground_truth.csv")
    asr_csv = os.path.join(CSV_OUTPUT_FOLDER, "asr_metrics.csv")
    asr_df = evaluate_asr_performance(gt_csv, trans_csv, asr_csv)

    # Step 3 — Load Blood Pressure Data
    df_bp = pd.read_csv(csv_path)
    csv_block = df_bp.to_csv(index=False)

    results = []
    for i, row in trans_df.iterrows():
        q_num = i + 1
        q_en = row["english_translation"]
        print(f"\n🔹 Q{q_num}: {q_en}")

        try:
            ans = ask_gpt(q_en, csv_block)
            ans_en = ans.get("answer", "").strip()
            ans_es = translate_to_spanish(ans_en)

            audio_file = os.path.join(AUDIO_OUTPUT_FOLDER, f"answer_{q_num}_es.wav")
            text_to_speech_spanish(ans_es, audio_file)

            results.append({
                "question_number": q_num,
                "audio_file_in": row["audio_file"],
                "spanish_question": row["spanish_transcription"],
                "english_question": q_en,
                "english_answer": ans_en,
                "spanish_answer": ans_es,
                "audio_answer_file": audio_file,
                "computed_fields": json.dumps(ans.get("computed_fields", {}))
            })
            print(f"✅ Completed Q{q_num}")

        except Exception as e:
            print(f"❌ Error Q{q_num}: {e}")

    # Step 4 — Save Final Results
    final_csv = os.path.join(CSV_OUTPUT_FOLDER, "final_pipeline_results.csv")
    pd.DataFrame(results).to_csv(final_csv, index=False)
    print(f"\n✅ All results saved to {final_csv}")
    return results

In [ ]:
# ================================================================
# 5️⃣ RUN
# ================================================================
csv_path = os.path.join(BASE_PATH, BP_DATA_FOLDER, "synthetic_bp_one_person.csv")
run_full_pipeline(csv_path, AUDIO_INPUT_FOLDER, audio_files)


100%|███████████████████████████████████████| 139M/139M [00:13<00:00, 10.6MiB/s]



🎯 STARTING SPANISH TRANSCRIPTION + TRANSLATION
🎧 Transcribing: /content/drive/MyDrive/health-tequity-case/Input_Audio_Files/q2_es.wav


100%|██████████| 470/470 [00:03<00:00, 138.64frames/s]



[1] q2_es.wav
🇪🇸 ¿Cuáles fueron mis valores de presión arterial durante la última semana?
🇬🇧 "What were my blood pressure values over the past week?"
🎧 Transcribing: /content/drive/MyDrive/health-tequity-case/Input_Audio_Files/q1_es.wav


100%|██████████| 458/458 [00:03<00:00, 145.44frames/s]


...

100%|██████████| 1574/1574 [00:05<00:00, 300.83frames/s]



[5] q6_es.wav
🇪🇸 ¿En qué día mi presión arterial excedió los niveles normales? Compare mi presión arterial promedio en la primera semana y la última semana de este mes. ¿Cuál fue mi presión arterial diastólica más baja este mes?
🇬🇧 On what day did my blood pressure exceed normal levels? Compare my average blood pressure in the first week and the last week of this month. What was my lowest diastolic blood pressure this month?
🎧 Transcribing: /content/drive/MyDrive/health-tequity-case/Input_Audio_Files/q5_es.wav


100%|██████████| 328/328 [00:02<00:00, 120.66frames/s]



[6] q5_es.wav
🇪🇸 ¿Cuál era mi presión arterial el 10 de octubre?
🇬🇧 What was my blood pressure on October 10?

✅ Transcriptions + translations saved to /content/drive/MyDrive/health-tequity-case/Data/csv_results/audio_translations.csv

🎯 Evaluating 6 files for ASR performance...

🎧 q1_es.wav → WER: 0.1111, CER: 0.0156, SER: 1
🎧 q2_es.wav → WER: 0.0, CER: 0.0, SER: 0
🎧 q3_es.wav → WER: 0.0, CER: 0.0, SER: 0
🎧 q4_es.wav → WER: 0.1, CER: 0.0175, SER: 1
🎧 q5_es.wav → WER: 0.0, CER: 0.0, SER: 0
🎧 q6_es.wav → WER: 0.0, CER: 0.0, SER: 0

✅ ASR metrics saved to: /content/drive/MyDrive/health-tequity-case/Data/csv_results/asr_metrics.csv

🔹 Q1: "What were my blood pressure values over the past week?"
🔊 Saved Spanish audio: /content/drive/MyDrive/health-tequity-case/Data/audio_out/answer_1_es.wav
✅ Completed Q1

🔹 Q2: "What are my systolic and diastolic blood pressures today?"
🔊 Saved Spanish audio: /content/drive/MyDrive/health-tequity-case/Data/audio_out/answer_2_es.wav
✅ Completed Q2

🔹 Q3: 

[{'question_number': 1,
  'audio_file_in': 'q2_es.wav',
  'spanish_question': '¿Cuáles fueron mis valores de presión arterial durante la última semana?',
  'english_question': '"What were my blood pressure values over the past week?"',
  'english_answer': 'Over the past week, your blood pressure readings were as follows: On 2025-10-09, your systolic was 160 mmHg and diastolic was 102 mmHg. On 2025-10-10, your systolic was 160 mmHg and diastolic was 101 mmHg. On 2025-10-11, your systolic was 152 mmHg and diastolic was 94 mmHg. On 2025-10-12, your systolic was 157 mmHg and diastolic was 98 mmHg. On 2025-10-13, your systolic was 144 mmHg and diastolic was 100 mmHg. On 2025-10-14, your systolic was 145 mmHg and diastolic was 91 mmHg. On 2025-10-15, your systolic was 124 mmHg and diastolic was 81 mmHg.',
  'spanish_answer': 'Durante la semana pasada, sus lecturas de presión arterial fueron las siguientes: El 9 de octubre de 2025, su presión sistólica fue de 160 mmHg y su presión diastólica 

In [ ]:
# !pip install vosk pydub
# !apt-get install ffmpeg


In [ ]:
!mkdir -p /content/vosk_models
!wget -q https://alphacephei.com/vosk/models/vosk-model-small-es-0.42.zip -O /content/vosk_models/vosk-model-small-es.zip
!unzip -q /content/vosk_models/vosk-model-small-es.zip -d /content/vosk_models/


In [ ]:
# ================================================================
# 6️⃣ OUTPUT AUDIO (TTS) ASR EVALUATION USING VOSK
# ================================================================
import os, json, wave
import pandas as pd
from vosk import Model, KaldiRecognizer
from jiwer import process_words
import Levenshtein
from pydub import AudioSegment

# ================================================================
# 🔧 Audio Conversion Helper
# ================================================================
def convert_to_wav(input_path, output_path, target_sr=16000):
    """
    Converts any audio file (MP3, M4A, WAV) to mono 16kHz RIFF WAV for Vosk.
    """
    try:
        audio = AudioSegment.from_file(input_path)
        audio = audio.set_frame_rate(target_sr).set_channels(1)
        audio.export(output_path, format="wav")
        return output_path
    except Exception as e:
        print(f"❌ Failed to convert {input_path}: {e}")
        return None

# ================================================================
# 🔊 Vosk Transcription
# ================================================================
def transcribe_with_vosk(audio_path, model_path="/content/vosk_models/vosk-model-small-es-0.42"):
    """
    Transcribes a Spanish audio file using Vosk offline ASR model.
    """
    if not os.path.exists(model_path):
        raise FileNotFoundError("❌ Vosk model not found. Please download and unzip it first.")

    model = Model(model_path)
    wf = wave.open(audio_path, "rb")
    if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getframerate() not in [16000, 22050, 44100]:
        raise ValueError(f"⚠️ Unsupported audio format in {audio_path}. Convert to mono 16kHz WAV first.")

    rec = KaldiRecognizer(model, wf.getframerate())
    rec.SetWords(True)

    result_text = ""
    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            part = json.loads(rec.Result())
            result_text += part.get("text", "") + " "
    part = json.loads(rec.FinalResult())
    result_text += part.get("text", "")
    wf.close()

    return result_text.strip()

# ================================================================
# 🧮 Evaluate TTS → Text using Vosk ASR
# ================================================================
def evaluate_output_asr(
    tts_csv,
    output_csv=os.path.join(CSV_OUTPUT_FOLDER, "output_asr_metrics.csv"),
    model_path="/content/vosk_models/vosk-model-small-es-0.42"
):
    """
    Evaluates TTS Spanish audio output using Vosk ASR model.
    Compares transcribed text vs. ground truth Spanish answers.
    """
    if not os.path.exists(tts_csv):
        raise FileNotFoundError(f"❌ Missing final results CSV: {tts_csv}")
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"❌ Vosk model not found at {model_path}. Download before running.")

    df = pd.read_csv(tts_csv)
    results = []

    print("\n🎯 Evaluating TTS → Spanish ASR transcription quality\n" + "="*60)
    for i, row in df.iterrows():
        gt = str(row["spanish_answer"])
        audio_file = row["audio_answer_file"]
        if not os.path.exists(audio_file):
            print(f"⚠️ Missing audio: {audio_file}")
            continue

        try:
            # Convert to proper WAV
            tmp_wav = os.path.join(AUDIO_OUTPUT_FOLDER, f"tmp_{i}.wav")
            converted_path = convert_to_wav(audio_file, tmp_wav)
            if not converted_path:
                print(f"⚠️ Could not convert {audio_file}, skipping...")
                continue

            # Transcribe with Vosk
            hyp = transcribe_with_vosk(converted_path, model_path)

            # Compute metrics
            measures = process_words(gt, hyp)
            wer_score = round(measures.wer, 4)
            subs, dels, ins = measures.substitutions, measures.deletions, measures.insertions
            cer = round(Levenshtein.distance(gt, hyp) / max(len(gt), 1), 4)
            ser = 0 if gt.strip() == hyp.strip() else 1

            results.append({
                "audio_file": os.path.basename(audio_file),
                "ground_truth": gt,
                "vosk_transcription": hyp,
                "WER": wer_score,
                "Substitutions": subs,
                "Deletions": dels,
                "Insertions": ins,
                "CER": cer,
                "SER": ser
            })

            print(f"🎧 {os.path.basename(audio_file)} → WER={wer_score}, CER={cer}, SER={ser}")

            # Clean up temp file
            os.remove(converted_path)

        except Exception as e:
            print(f"❌ Error processing {audio_file}: {e}")

    out_df = pd.DataFrame(results)
    out_df.to_csv(output_csv, index=False)
    print(f"\n✅ Output ASR evaluation saved to: {output_csv}")
    return out_df


In [ ]:
# Evaluate the Spanish TTS outputs using Vosk ASR
final_results_csv = os.path.join(CSV_OUTPUT_FOLDER, "final_pipeline_results.csv")
output_asr_metrics = evaluate_output_asr(final_results_csv)
display(output_asr_metrics.head())




🎯 Evaluating TTS → Spanish ASR transcription quality
🎧 answer_1_es.wav → WER=1.0629, CER=0.9241, SER=1
🎧 answer_2_es.wav → WER=0.8889, CER=0.7604, SER=1
🎧 answer_3_es.wav → WER=0.5, CER=0.3179, SER=1
🎧 answer_4_es.wav → WER=0.3462, CER=0.1712, SER=1
🎧 answer_5_es.wav → WER=0.7843, CER=0.7692, SER=1
🎧 answer_6_es.wav → WER=0.9231, CER=1.0455, SER=1

✅ Output ASR evaluation saved to: /content/drive/MyDrive/health-tequity-case/Data/csv_results/output_asr_metrics.csv


,audio_file,ground_truth,vosk_transcription,WER,Substitutions,Deletions,Insertions,CER,SER
0,answer_1_es.wav,"Durante la semana pasada, sus lecturas de pres...",durante la semana pasada sus lecturas de presi...,1.0629,92,1,76,0.9241,1
1,answer_2_es.wav,Su presión arterial sistólica hoy es de 110 mm...,su presión arterial histórica hoy es de ciento...,0.8889,7,0,9,0.7604,1
2,answer_3_es.wav,"Para una mujer de 68 años, se considera que la...",para una mujer de sesenta y ocho años se consi...,0.5000,15,0,9,0.3179,1
3,answer_4_es.wav,Los valores de la presión arterial muestran un...,los valores de la presión arterial muestran un...,0.3462,31,0,5,0.1712,1
4,answer_5_es.wav,Su presión arterial superó los niveles normale...,su presión arterial superó los niveles normale...,0.7843,39,0,41,0.7692,1


In [ ]:
# ================================================================
# 6️⃣ OUTPUT AUDIO (TTS) ASR EVALUATION USING WHISPER
# ================================================================
import whisper, re, unicodedata, Levenshtein, pandas as pd
from jiwer import process_words

def normalize_text(text):
    """Lowercase, strip accents, remove punctuation for fair WER/CER."""
    text = text.lower()
    text = ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )
    text = re.sub(r'[^a-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def evaluate_output_asr_whisper(
    tts_csv,
    output_csv=os.path.join(CSV_OUTPUT_FOLDER, "output_asr_metrics_whisper.csv"),
    model_size="base"
):
    """
    Evaluate the Spanish TTS audios using Whisper.
    Computes WER, CER, and SER against ground truth Spanish answers.
    """
    if not os.path.exists(tts_csv):
        raise FileNotFoundError(f"❌ Missing final results CSV: {tts_csv}")

    print(f"🎯 Loading Whisper ({model_size}) model ...")
    model = whisper.load_model(model_size)

    df = pd.read_csv(tts_csv)
    results = []

    print("\n🎧 Evaluating Spanish TTS output audios\n" + "="*60)
    for i, row in df.iterrows():
        gt = str(row["spanish_answer"]).strip()
        audio_file = row["audio_answer_file"]
        if not os.path.exists(audio_file):
            print(f"⚠️ Missing audio file: {audio_file}")
            continue

        try:
            # Transcribe with Whisper
            result = model.transcribe(audio_file, language="es", task="transcribe", verbose=False)
            hyp = result["text"].strip()

            # Normalize both texts
            gt_norm = normalize_text(gt)
            hyp_norm = normalize_text(hyp)

            # Compute metrics
            measures = process_words(gt_norm, hyp_norm)
            wer_score = round(measures.wer, 4)
            subs, dels, ins = measures.substitutions, measures.deletions, measures.insertions
            cer = round(Levenshtein.distance(gt_norm, hyp_norm) / max(len(gt_norm), 1), 4)
            ser = 0 if gt_norm == hyp_norm else 1

            results.append({
                "audio_file": os.path.basename(audio_file),
                "ground_truth": gt,
                "whisper_transcription": hyp,
                "WER": wer_score,
                "Substitutions": subs,
                "Deletions": dels,
                "Insertions": ins,
                "CER": cer,
                "SER": ser
            })

            print(f"✅ {os.path.basename(audio_file)} → WER={wer_score}, CER={cer}, SER={ser}")

        except Exception as e:
            print(f"❌ Error processing {audio_file}: {e}")

    out_df = pd.DataFrame(results)
    out_df.to_csv(output_csv, index=False)
    print(f"\n✅ Whisper ASR evaluation saved to: {output_csv}")
    return out_df


In [ ]:
final_results_csv = os.path.join(CSV_OUTPUT_FOLDER, "final_pipeline_results.csv")
output_asr_metrics = evaluate_output_asr_whisper(final_results_csv, model_size="base")
display(output_asr_metrics.head())


🎯 Loading Whisper (base) model ...

🎧 Evaluating Spanish TTS output audios


100%|██████████| 7260/7260 [01:54<00:00, 63.34frames/s]


✅ answer_1_es.wav → WER=0.327, CER=0.2259, SER=1


100%|██████████| 885/885 [00:03<00:00, 229.36frames/s]


✅ answer_2_es.wav → WER=0.6111, CER=0.4526, SER=1


...

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['whisper_transcription'].value_counts()
    for x_label, grp in _df_20.groupby('ground_truth')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('ground_truth')
_ = plt.ylabel('whisper_transcription')

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_21['audio_file'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_21, x='WER', y='audio_file', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_22['ground_truth'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_22, x='WER', y='ground_truth', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_23['whisper_transcription'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_23, x='WER', y='whisper_transcription', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_24['audio_file'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_24, x='Substitutions', y='audio_file', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)